In [87]:
# tests.ipynb
import numpy as np
import matplotlib.pyplot as plt

from qdc.diffuser.diffuser_sim import DiffuserSimulation, Field
from qdc.diffuser.utils import prop_farfield_fft, backprop_farfield_fft, ft2, ift2, phase_screen_diff
from qdc.diffuser.diffuser_result import DiffuserResult

show_all = False
# -------------
# 1) Setup simulation
# -------------
sim = DiffuserSimulation(
    Nx=1024, Ny=1024,
    Lx=2.5e-3, Ly=2.5e-3,
    wl0=808e-9,
    Dwl=30e-9,
    Nwl=5,
    waist=50e-6,
    focal_length=250e-3,
    diffuser_angle=4 * 2*np.pi/360
)

# -------------
# 2) Initial Gaussian at detection
# -------------
field_det = sim.make_detection_gaussian(sim.wl_center)
if show_all or False:
    field_det.show(title="Initial Gaussian at Detection Plane (center WL)")
print(f'{(np.abs(field_det.E) ** 2).sum()=}')

# -------------
# 3) Back-propagate to crystal plane
# -------------
field_crystal = sim.make_detection_gaussian(sim.wl_center)
field_crystal = backprop_farfield_fft(field_crystal, sim.f)  # or use the function from utils
if show_all or False:
    field_crystal.show(title=f"Field Intensity at Crystal Plane f={sim.f}")

# -------------
# 4) Single diffuser phase at lam_center
# -------------
phase_ref = phase_screen_diff(sim.x, sim.y, sim.wl_center, sim.diffuser_angle)
if show_all or False:
    fig, ax = plt.subplots()
    pcm = ax.imshow(phase_ref, extent=[sim.x[0]*1e3, sim.x[-1]*1e3, sim.y[0]*1e3, sim.y[-1]*1e3],
                    cmap='twilight', origin='lower')
    fig.colorbar(pcm, ax=ax, label='Phase [rad]')
    ax.set_title("Single Diffuser Phase (lam_center)")
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("y [mm]")
    fig.show()


# -------------
# 5) k-space amplitude after applying diffuser (for a given WL)
#    Let's pick lam = lam_center for demonstration
# -------------
lam = sim.wl_center
sc_field = Field(sim.x, sim.y, lam, field_crystal.E.copy())

# scale the diffuser phase for this lam = lam_center => scale=1
phase_lam = (sim.wl_center / lam) * phase_ref
sc_field.E *= np.exp(1j * phase_lam)

# Now do an ft2 to see k-space amplitude
from qdc.diffuser.utils import ft2
G, f_x, f_y = ft2(sc_field.E, sc_field.x, sc_field.y)
Ik = np.abs(G)**2
if show_all or False:
    fig, ax = plt.subplots()
    im = ax.imshow(Ik, origin='lower', cmap='inferno')
    fig.colorbar(im, ax=ax, label='|G(kx,ky)|^2')
    ax.set_title("k-space amplitude after diffuser")
    ax.set_xlabel("freq_x [index]")
    ax.set_ylabel("freq_y [index]")
    fig.show()

# -------------
# 6) Forward-propagate to detection => final speckle at that wavelength
# -------------
from qdc.diffuser.utils import prop_farfield_fft
final_field = prop_farfield_fft(sc_field, sim.f)
if show_all or True:
    final_field.show(title=f"Speckle at Detection (WL={lam*1e9:.1f}nm)")

# -------------
# 7) Full Incoherent Sum Over All Wavelengths
# -------------
I_sum = sim.run_SPDC_simulation()  # uses the single diffuser approach inside
res = DiffuserResult(intensity_map=I_sum, wavelengths=None)
if show_all or True:
    fig, ax = plt.subplots()
    im = ax.imshow(I_sum, origin='lower',
                   extent=[sim.x[0]*1e3, sim.x[-1]*1e3, sim.y[0]*1e3, sim.y[-1]*1e3],
                   cmap='viridis')
    fig.colorbar(im, ax=ax, label='Intensity')
    ax.set_title("Incoherent Sum of Speckle (All WLs)")
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("y [mm]")
    fig.show()

print("Final result:", res)
print("Contrast (full frame):", res.compute_contrast())

(np.abs(field_det.E) ** 2).sum()=1.0000000000000002
Final result: <SPDCResult shape=(1024, 1024) min=2.96e-17, max=1.31e-11>
Contrast (full frame): 1.5056539193951297
